In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import VLM
from wind import WindField
from scipy.interpolate import UnivariateSpline, LinearNDInterpolator, NearestNDInterpolator
from scipy.io import savemat
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numba as nb
import copy
import pyvista as pv


# # def visualize_wake_development(quad_propeller_mesh, wake_system, propeller_key="Propeller_1", 
# #                             show_bound=True, time_step=0, save_plot=False):
# #     """
# #     Visualize wake development at a specific time step with different colors for wake age.
    
# #     Args:
# #         quad_propeller_mesh: The mesh system
# #         wake_system: The wake system
# #         propeller_key: Which propeller to plot
# #         show_bound: Whether to show bound vortex system
# #         time_step: Current time step for labeling
# #         save_plot: Whether to save the plot to file
# #     """
# #     fig = plt.figure(figsize=(15, 10))
# #     ax = fig.add_subplot(111, projection='3d')
    
# #     # Color map for wake age
# #     wake_colors = plt.cm.viridis(np.linspace(0, 1, 10))  # 10 different colors for wake age
    
# #     # Plot bound system if requested
# #     if show_bound:
# #         propeller_data = quad_propeller_mesh[propeller_key]
# #         for blade_key, blade_data in propeller_data['Blades'].items():
# #             # Plot bound vortex rings
# #             for panel_index, vortex_ring in blade_data['Vortex Rings'].items():
# #                 vertices = np.array(vortex_ring['Vertices'])
                
# #                 # Plot bound vertices
# #                 ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
# #                         color='blue', Alpha=0.5, s=50)
                
# #                 # Connect bound vertices
# #                 for i in range(4):
# #                     start = vertices[i]
# #                     end = vertices[(i + 1) % 4]
# #                     ax.plot([start[0], end[0]], 
# #                         [start[1], end[1]], 
# #                         [start[2], end[2]], 
# #                         'b-', alpha=0.5)
    
# #     # Plot wake system with color-coded age
# #     wake_data = wake_system[propeller_key]
# #     for blade_key, blade_data in wake_data.items():
# #         for wake_index, wake_panel in blade_data.items():
# #             if 'Vortex Rings' in wake_panel and wake_panel['Vortex Rings']['Vertices'] is not None:
# #                 vertices = np.array(wake_panel['Vortex Rings']['Vertices'])
                
# #                 # Determine wake age and corresponding color
# #                 wake_age = wake_index[0]  # Using first index as age indicator
# #                 color_index = min(wake_age, len(wake_colors)-1)
# #                 wake_color = wake_colors[color_index]
                
# #                 # Plot wake vertices
# #                 ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
# #                         color=wake_color, alpha=0.7, s=30)
                
# #                 # Connect wake vertices with age-colored lines
# #                 for i in range(4):
# #                     start = vertices[i]
# #                     end = vertices[(i + 1) % 4]
# #                     ax.plot([start[0], end[0]], 
# #                         [start[1], end[1]], 
# #                         [start[2], end[2]], 
# #                         color=wake_color, alpha=0.7)
                
# #                 # Plot wake control points if they exist
# #                 if wake_panel['Control Points'] is not None:
# #                     cp = wake_panel['Control Points']
# #                     ax.scatter(cp[0], cp[1], cp[2], color='red', s=50)
    
# #     # Add legend and labels
# #     ax.scatter([], [], [], c='blue', alpha=0.5, s=50, label='Bound Vertices')
# #     ax.scatter([], [], [], c='red', s=50, label='Wake Control Points')
    
# #     # Set labels and title
# #     ax.set_xlabel('X')
# #     ax.set_ylabel('Y')
# #     ax.set_zlabel('Z')
# #     ax.set_title(f'Wake Development - {propeller_key} (Time Step: {time_step})')
    
# #     # Add colorbar to show wake age
# #     sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
# #     sm.set_array([])
# #     cbar = plt.colorbar(sm)
# #     cbar.set_label('Wake Age')
    
# #     # Set equal aspect ratio
# #     max_range = np.array([
# #         ax.get_xlim3d()[1] - ax.get_xlim3d()[0],
# #         ax.get_ylim3d()[1] - ax.get_ylim3d()[0],
# #         ax.get_zlim3d()[1] - ax.get_zlim3d()[0]
# #     ]).max() / 2.0
    
# #     mid_x = (ax.get_xlim3d()[1] + ax.get_xlim3d()[0]) / 2.0
# #     mid_y = (ax.get_ylim3d()[1] + ax.get_ylim3d()[0]) / 2.0
# #     mid_z = (ax.get_zlim3d()[1] + ax.get_zlim3d()[0]) / 2.0
# #     ax.set_xlim3d([mid_x - max_range, mid_x + max_range])
# #     ax.set_ylim3d([mid_y - max_range, mid_y + max_range])
# #     ax.set_zlim3d([mid_z - max_range, mid_z + max_range])
    
# #     # Save plot if requested
# #     if save_plot:
# #         plt.savefig(f'wake_development_step_{time_step}.png')
    
# #     plt.show()

# # def store_wake_state(propeller_mesh, wake_system):
# #     """Store current state of wake system"""
# #     state = {
# #         'bound_vertices': [],
# #         'wake_vertices': [],
# #         'control_points': []
# #     }

# #     # Store bound vortex rings
# #     for prop_key, prop_data in propeller_mesh.items():
# #         for blade_key, blade_data in prop_data['Blades'].items():
# #             for vortex_ring in blade_data['Vortex Rings'].values():
# #                 state['bound_vertices'].append(np.array(vortex_ring['Vertices']))

# #     # Store wake panels
# #     for prop_key, prop_data in wake_system.items():
# #         for blade_key, blade_data in prop_data.items():
# #             for wake_panel in blade_data.values():
# #                 if wake_panel['Vortex Rings']['Vertices'] is not None:
# #                     state['wake_vertices'].append(np.array(wake_panel['Vortex Rings']['Vertices']))
# #                 if wake_panel['Control Points'] is not None:
# #                     state['control_points'].append(np.array(wake_panel['Control Points']))

# #     return state

# # def store_wake_state(propeller_mesh, wake_system):
#     """Store current state of wake system"""
#     state = {
#         'bound_vertices': [],
#         'wake_vertices': [],
#         'control_points': []
#     }

#     # Debug prints
#     print("\nStoring wake state:")
#     print("Number of propellers:", len(propeller_mesh))

#     # Store bound vortex rings
#     for prop_key, prop_data in propeller_mesh.items():
#         print(f"\nProcessing {prop_key}")
#         for blade_key, blade_data in prop_data['Blades'].items():
#             print(f"Processing {blade_key}, number of vortex rings:", len(blade_data['Vortex Rings']))
#             for vr_idx, vortex_ring in blade_data['Vortex Rings'].items():
#                 vertices = np.array(vortex_ring['Vertices'])
#                 print(f"Vortex ring {vr_idx} vertices shape:", vertices.shape)
#                 state['bound_vertices'].append(vertices)

#     # Store wake panels
#     print("\nProcessing wake system:")
#     for prop_key, prop_data in wake_system.items():
#         print(f"\nPropeller {prop_key} wake:")
#         for blade_key, blade_data in prop_data.items():
#             print(f"Blade {blade_key} wake panels:", len(blade_data))
#             for wake_idx, wake_panel in blade_data.items():
#                 if wake_panel['Vortex Rings']['Vertices'] is not None:
#                     vertices = np.array(wake_panel['Vortex Rings']['Vertices'])
#                     print(f"Wake panel {wake_idx} vertices shape:", vertices.shape)
#                     state['wake_vertices'].append(vertices)
#                 if wake_panel['Control Points'] is not None:
#                     cp = np.array(wake_panel['Control Points'])
#                     print(f"Control point {wake_idx} shape:", cp.shape)
#                     state['control_points'].append(cp)

#     print("\nFinal state sizes:")
#     print("Bound vertices:", len(state['bound_vertices']))
#     print("Wake vertices:", len(state['wake_vertices']))
#     print("Control points:", len(state['control_points']))

#     return state

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------_#
# wake_states = []  # List to store states at each time step

# Initialize geometry and mesh
# propeller_geometry = PropellerGeometry(
#     airfoil_distribution_file='DJI9443_airfoils.csv',
#     chorddist_file='DJI9443_chorddist.csv',
#     pitchdist_file='DJI9443_pitchdist.csv',
#     sweepdist_file='DJI9443_sweepdist.csv',
#     heightdist_file='DJI9443_heightdist.csv',
#     R_tip=0.11938,
#     R_hub=0.00624,
#     num_blades=2
# )


# Initialize and generate mesh as before
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
print('Propeller Loaded Succesfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Succesfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg

inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates



# Initialize the UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)

wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

time_points = []
normalized_forces = []
rho = 1.071778   
# 

mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)

rpm = 45
zerp = np.array([0, 0, 0])
com_position = np.array([300, 135, 50])
wind_func = WindField.update_wind_function(wind_field, com_position)
wind_velocity = -wind_func(zerp)


# Initialize arrays to store results
x_positions = []
wind_velocities = []

# Initial constants
zerp = np.array([0, 0, 0])
y_position = 135
z_position = 50

# Loop through x positions from 150 to 450 in steps of 5
for x_position in range(150, 451, 5):  # 451 to include 450
    # Create com_position with current x value
    com_position = np.array([x_position, y_position, z_position])
    
    # Calculate wind function and velocity
    wind_func = WindField.update_wind_function(wind_field, com_position)
    wind_velocity = -wind_func(zerp)
    
    # Store results
    x_positions.append(x_position)
    wind_velocities.append(wind_velocity)
    
    # Optional: Print progress (can be removed)
    print(f"x: {x_position}, wind_velocity: {wind_velocity}")
x_positions = np.array(x_positions)
# Method 2: Save as CSV
# Convert list of wind velocities to a proper 2D numpy array
if wind_velocities:
    wind_velocities = np.array(wind_velocities)
    
    # Check if wind_velocities has the right shape
    if wind_velocities.ndim == 1:
        # If it's 1D, reshape or split into components
        # Assuming wind_velocity is a 3-component vector
        wind_velocities = wind_velocities.reshape(-1, 3)
else:
    print("Warning: No wind velocities were collected!")

# Example: Save to file (choose one of these methods)
# Method 1: Save as numpy arrays
np.save('x_positions.npy', x_positions)
np.save('wind_velocities.npy', wind_velocities)

# Method 2: Save as CSV
import pandas as pd

# Create DataFrame depending on the shape of wind_velocities
if wind_velocities.ndim >= 2 and wind_velocities.shape[1] >= 3:
    df = pd.DataFrame({
        'x_position': x_positions,
        'wind_velocity_x': wind_velocities[:, 0],
        'wind_velocity_y': wind_velocities[:, 1],
        'wind_velocity_z': wind_velocities[:, 2]
    })
else:
    # Fallback if wind_velocities doesn't have expected shape
    df = pd.DataFrame({
        'x_position': x_positions,
        'wind_velocity': wind_velocities.flatten()
    })

df.to_csv('wind_velocities.csv', index=False)

print('wind',wind_velocity )
n_revs = 2
n_steps_rev = 80
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (rpm * n_steps_rev)

omega_dict = {  # Motor speed initialization
    'Propeller_1': np.array([0, 0, rpm]),
    'Propeller_2': np.array([0, 0, -rpm]),
    'Propeller_3': np.array([0, 0, -rpm]),
    'Propeller_4': np.array([0, 0, rpm])
}

for time_step in range(1, 2):

    # Step 1: Calculate forces and moments using UVLM
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body, 
        omega=omega_dict,
        wind_field=wind_field,
        com_position=com_position,
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )

    # forces_and_moments = uvlm_solver.viscous_coupling(
    #     quad_propeller_mesh=quad_propeller_mesh,
    #     omega_dict=omega_dict,
    #     body_velocity=dynamics.velocity_body, 
    #     wind_field=wind_field,
    #     com_position=com_position,
    #     time_step=time_step,
    #     dt=dt,
    #     roll=dynamics.angles[0],
    #     pitch=dynamics.angles[1],
    #     yaw=dynamics.angles[2],
    #     rho=rho,
    # )


    # print("Forces:", forces_and_moments['total_force'])
    # print("Forces:", forces_and_moments['total_moment'])
    print('force', forces_and_moments['Propeller_1']['force'])
    print('momnet', forces_and_moments['Propeller_1']['moment'])

    print("Time step:", time_step)
   

    # # Step 2: Update mesh geometry for propeller rotation
    # SixDOFDynamics.update_mesh_transformations(
    #     propeller_mesh=quad_propeller_mesh,
    #     dynamics=dynamics,
    #     dt=dt,
    #     omega_dict=omega_dict
    # )

    # # Step 3: Update wake
    # uvlm_solver.update_wake(
    #     propeller_mesh=quad_propeller_mesh,
    #     time_step=time_step,
    #     dt=dt,
    #     body_velocity=dynamics.velocity_body,
    #     omega_dict=omega_dict
    # )
    
    # uvlm_solver.store_thrust(forces_and_moments, time_step)
    
    # uvlm_solver.plot_wake_system(quad_propeller_mesh, uvlm_solver.wake_system, 
    #             "Propeller_1", azimuth=0, elevation=45)

#     fixed_radial_index = 3 
#     uvlm_solver.plot_span_gamma_distribution(quad_propeller_mesh, fixed_radial_index)


#     fixed_radial_index = 3 
#     uvlm_solver.plot_chord_gamma_distribution(quad_propeller_mesh, fixed_radial_index)
#     uvlm_solver.plot_detailed_gamma_distribution(quad_propeller_mesh)

#     # uvlm_solver.plot_wake_system(quad_propeller_mesh, uvlm_solver.wake_system, 
#     #                     "Propeller_1", azimuth=0, elevation=30)

#     uvlm_solver.plot_blades_pressure_top_view(
#     quad_propeller_mesh,
#     propeller_key="Propeller_1",
#     omega=omega_dict["Propeller_1"], 
#     body_velocity=initial_body_velocity,   
#     rho=rho
# )
# uvlm_solver.plot_wake_system(quad_propeller_mesh, uvlm_solver.wake_system, 
#                         "Propeller_1", azimuth=0, elevation=0)


# uvlm_solver.plot_thrust_history()   






In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from vlm import VLM
import numpy as np
import pandas as pd
import math

# Initialize geometry and mesh
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

print('Initializing propeller mesh...')
propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh generated successfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg
inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]
])  # Simple diagonal inertia matrix

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

# Initialize the UVLM solver
print('Initializing UVLM solver...')
uvlm_solver = VLM(quad_propeller_mesh)

# Define RPM, wind magnitude, and angle values to test
rpm_values = [38, 39, 40, 41, 42, 43, 44, 45, 46]  # 20, 25, 30, 35, 40, 45
wind_magnitudes = np.arange(0, 13, 3)
angles_degrees = np.arange(-25, 26, 5)

# Common simulation parameters
n_revs = 2
n_steps_rev = 80
com_position = np.array([0, 0, 0])
rho = 1.071778

# Instead of creating a wind field class, we'll directly use the wind vector
# This is because the VLM code seems to expect a wind vector directly

# Create dictionary to store all simulation results
all_results = []

# Loop through each RPM value
for rpm in rpm_values:
    print(f"\n{'-'*70}")
    print(f"Running simulations for RPM: {rpm}")
    print(f"{'-'*70}")
    
    # Calculate time step based on RPM
    dt = 2*np.pi / (rpm * n_steps_rev)
    
    # Set motor speeds for each propeller
    omega_dict = {
        'Propeller_1': np.array([0, 0, rpm]),
        'Propeller_2': np.array([0, 0, -rpm]),
        'Propeller_3': np.array([0, 0, -rpm]),
        'Propeller_4': np.array([0, 0, rpm])
    }
    
    # Loop through each wind magnitude
    for wind_magnitude in wind_magnitudes:
        print(f"\nTesting wind magnitude: {wind_magnitude}")
        
        # Loop through each incidence angle
        for angle_deg in angles_degrees:
            # Convert angle to radians
            angle_rad = math.radians(angle_deg)
            
            # Calculate wind vector components
            # At 0 degrees, wind is purely in x-direction
            wind_x = wind_magnitude * math.cos(angle_rad)
            wind_y = 0
            wind_z = wind_magnitude * math.sin(angle_rad)
            
            wind_vector = np.array([wind_x, wind_y, wind_z])
            
            # Calculate wind x-component (Vmag * cos(angle))
            wind_x_component = wind_magnitude * math.cos(angle_rad)
            
            print(f"  Angle: {angle_deg}°, Wind vector: [{wind_x:.2f}, {wind_y:.2f}, {wind_z:.2f}]")
            
            # Use the wind vector directly without creating a wind field object
            # Run simulation for a single time step
            forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
                propeller_mesh=quad_propeller_mesh,
                dt=dt,
                rho=rho,
                time_step=1,  # Using a single time step
                body_velocity=dynamics.velocity_body, 
                omega=omega_dict,
                wind_field=wind_vector,  # Pass the wind vector directly
                com_position=com_position,
                roll=dynamics.angles[0],
                pitch=dynamics.angles[1],
                yaw=dynamics.angles[2]
            )
            
            # Calculate total force and moment from all propellers
            total_force = np.zeros(3)
            total_moment = np.zeros(3)
            
            # Extract forces and moments for each propeller
            propeller_results = {}
            for propeller_key in forces_and_moments:
                propeller_force = forces_and_moments[propeller_key]['force']
                propeller_moment = forces_and_moments[propeller_key]['moment']
                
                # Add to total
                total_force += propeller_force
                total_moment += propeller_moment
                
                # Store propeller-specific results
                propeller_results[propeller_key] = {
                    'Fx': propeller_force[0],
                    'Fy': propeller_force[1],
                    'Fz': propeller_force[2],
                    'Qx': propeller_moment[0],
                    'Qy': propeller_moment[1],
                    'Qz': propeller_moment[2]
                }
            
            # Store results for this simulation condition
            result_row = {
                'RPM': rpm,
                'WindMagnitude': wind_magnitude,
                'AngleDegrees': angle_deg,
                'WindX': wind_x,
                'WindY': wind_y,
                'WindZ': wind_z,
                'VmagCosAngle': wind_x_component,
                'Fx': total_force[0],
                'Fy': total_force[1],
                'Fz': total_force[2],
                'Qx': total_moment[0],
                'Qy': total_moment[1],
                'Qz': total_moment[2]
            }
            
            # Add propeller-specific results
            for propeller_key, prop_data in propeller_results.items():
                for force_key, force_value in prop_data.items():
                    result_row[f'{propeller_key}_{force_key}'] = force_value
            
            # Add this row to the results
            all_results.append(result_row)
            
            print(f"    Total Force: [{total_force[0]:.4f}, {total_force[1]:.4f}, {total_force[2]:.4f}]")
            print(f"    Total Moment: [{total_moment[0]:.4f}, {total_moment[1]:.4f}, {total_moment[2]:.4f}]")

# Create DataFrame from all results
results_df = pd.DataFrame(all_results)

# Save the results to CSV
print("\nSaving results to CSV file...")
results_df.to_csv('rpm_wind_simulation_results_correct_3.csv', index=False)
print("Results saved to rpm_wind_simulation_results.csv")

print("Summary files created.")
print("All simulations completed successfully!")

In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from vlm import VLM
import numpy as np
import pandas as pd
import math

# Initialize geometry and mesh
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

print('Initializing propeller mesh...')
propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh generated successfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg
inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]
])  # Simple diagonal inertia matrix

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

# Initialize the UVLM solver
print('Initializing UVLM solver...')
uvlm_solver = VLM(quad_propeller_mesh)

# Define RPM, wind magnitude, and angle values to test
rpm_values = [45]  # 20, 25, 30, 35, 40, 45
wind_magnitudes = [0, 2, 4, 6, 8, 10, 12]
angles_degrees = [-20, -15, -10, -5, 0, 5, 10, 15, 20]  # -20, -15, -10, -5, 0, 5, 10, 15, 20

# Common simulation parameters
n_revs = 1
n_steps_rev = 10
n_steps = n_revs * n_steps_rev
com_position = np.array([0, 0, 0])
rho = 1.071778

# Create dictionary to store all simulation results
all_results = []

# Loop through each RPM value
for rpm in rpm_values:
    print(f"\n{'-'*70}")
    print(f"Running simulations for RPM: {rpm}")
    print(f"{'-'*70}")
    
    # Calculate time step based on RPM
    dt = 2*np.pi / (rpm * n_steps_rev)
    
    # Set motor speeds for each propeller
    omega_dict = {
        'Propeller_1': np.array([0, 0, rpm]),
        'Propeller_2': np.array([0, 0, -rpm]),
        'Propeller_3': np.array([0, 0, -rpm]),
        'Propeller_4': np.array([0, 0, rpm])
    }
    
    # Loop through each wind magnitude
    for wind_magnitude in wind_magnitudes:
        print(f"\nTesting wind magnitude: {wind_magnitude}")
        
        # Loop through each incidence angle
        for angle_deg in angles_degrees:
            # Convert angle to radians
            angle_rad = math.radians(angle_deg)
            
            # Calculate wind vector components
            # At 0 degrees, wind is purely in x-direction
            wind_x = wind_magnitude * math.cos(angle_rad)
            wind_y = 0
            wind_z = wind_magnitude * math.sin(angle_rad)
            
            wind_vector = np.array([wind_x, wind_y, wind_z])
            
            # Calculate wind x-component (Vmag * cos(angle))
            wind_x_component = wind_magnitude * math.cos(angle_rad)
            
            print(f"  Angle: {angle_deg}°, Wind vector: [{wind_x:.2f}, {wind_y:.2f}, {wind_z:.2f}]")
            
            # Inner loop for azimuth data (multiple time steps per angle)
            for time_step in range(1, n_steps_rev+1):
                print(f"    Time step: {time_step}/{n_steps_rev}")
                
                # Calculate current azimuth angle
                azimuth_angle = (time_step - 1) * 2 * np.pi / n_steps_rev
                azimuth_deg = math.degrees(azimuth_angle)
                
                # Apply cyclic pitch by modulating the wind angle
                # This creates a sinusoidal variation in the effective wind angle
                cyclic_amplitude_deg = 3.0  # ±3 degrees
                cyclic_angle_offset = cyclic_amplitude_deg * math.sin(azimuth_angle)
                
                # Calculate effective wind angle with cyclic pitch
                effective_angle_deg = angle_deg + cyclic_angle_offset
                effective_angle_rad = math.radians(effective_angle_deg)
                
                # Calculate modified wind vector components with cyclic pitch
                effective_wind_x = wind_magnitude * math.cos(effective_angle_rad)
                effective_wind_y = 0
                effective_wind_z = wind_magnitude * math.sin(effective_angle_rad)
                
                effective_wind_vector = np.array([effective_wind_x, effective_wind_y, effective_wind_z])
                
                print(f"Base angle: {angle_deg}°, Cyclic offset: {cyclic_angle_offset:.2f}°, Effective angle: {effective_angle_deg:.2f}°")
                
                # Step 1: Calculate forces and moments using UVLM
                forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
                    propeller_mesh=quad_propeller_mesh,
                    dt=dt,
                    rho=rho,
                    time_step=time_step,
                    body_velocity=dynamics.velocity_body, 
                    omega=omega_dict,
                    wind_field=effective_wind_vector,  # Use the cyclic-modified wind vector
                    com_position=com_position,
                    roll=dynamics.angles[0],
                    pitch=dynamics.angles[1],
                    yaw=dynamics.angles[2]
                )
                
                # Calculate total force and moment from all propellers
                total_force = np.zeros(3)
                total_moment = np.zeros(3)
                
                # Extract forces and moments for each propeller
                propeller_results = {}
                for propeller_key in forces_and_moments:
                    propeller_force = forces_and_moments[propeller_key]['force']
                    propeller_moment = forces_and_moments[propeller_key]['moment']
                    
                    # Add to total
                    total_force += propeller_force
                    total_moment += propeller_moment
                    
                    # Store propeller-specific results
                    propeller_results[propeller_key] = {
                        'Fx': propeller_force[0],
                        'Fy': propeller_force[1],
                        'Fz': propeller_force[2],
                        'Qx': propeller_moment[0],
                        'Qy': propeller_moment[1],
                        'Qz': propeller_moment[2]
                    }
                
                # Store results for this simulation condition
                result_row = {
                    'RPM': rpm,
                    'WindMagnitude': wind_magnitude,
                    'AngleDegrees': angle_deg,
                    'TimeStep': time_step,
                    'AzimuthAngle': azimuth_deg,
                    'WindX': wind_x,
                    'WindY': wind_y,
                    'WindZ': wind_z,
                    'VmagCosAngle': wind_x_component,
                    'Fx': total_force[0],
                    'Fy': total_force[1],
                    'Fz': total_force[2],
                    'Qx': total_moment[0],
                    'Qy': total_moment[1],
                    'Qz': total_moment[2]
                }
                
                # Add propeller-specific results
                for propeller_key, prop_data in propeller_results.items():
                    for force_key, force_value in prop_data.items():
                        result_row[f'{propeller_key}_{force_key}'] = force_value
                
                # Add this row to the results
                all_results.append(result_row)
                
                print(f"      Total Force: [{total_force[0]:.4f}, {total_force[1]:.4f}, {total_force[2]:.4f}]")
                print(f"      Total Moment: [{total_moment[0]:.4f}, {total_moment[1]:.4f}, {total_moment[2]:.4f}]")
                
                # Step 2: Update mesh transformations for next time step
                SixDOFDynamics.update_mesh_transformations(
                    propeller_mesh=quad_propeller_mesh,
                    dynamics=dynamics,
                    dt=dt,
                    omega_dict=omega_dict
                )

# Create DataFrame from all results
results_df = pd.DataFrame(all_results)

# Save the results to CSV
print("\nSaving results to CSV file...")
results_df.to_csv('rpm_wind_simulation_results_with_azimuth.csv', index=False)
print("Results saved to rpm_wind_simulation_results_with_azimuth.csv")

# Also create separate CSV files with summary data for easier analysis
print("Creating summary CSV files...")

# Group by RPM and create CSV with average forces/moments by RPM
rpm_summary = results_df.groupby('RPM')[['Fx', 'Fy', 'Fz', 'Qx', 'Qy', 'Qz']].mean()
rpm_summary.to_csv('summary_by_rpm.csv')

# Group by wind magnitude and create CSV with average forces/moments by wind magnitude
wind_summary = results_df.groupby('WindMagnitude')[['Fx', 'Fy', 'Fz', 'Qx', 'Qy', 'Qz']].mean()
wind_summary.to_csv('summary_by_wind_magnitude.csv')

# Group by angle and create CSV with average forces/moments by angle
angle_summary = results_df.groupby('AngleDegrees')[['Fx', 'Fy', 'Fz', 'Qx', 'Qy', 'Qz']].mean()
angle_summary.to_csv('summary_by_angle.csv')

# Create summary by azimuth angle (across all conditions)
azimuth_summary = results_df.groupby('AzimuthAngle')[['Fx', 'Fy', 'Fz', 'Qx', 'Qy', 'Qz']].mean()
azimuth_summary.to_csv('summary_by_azimuth.csv')

# Create detailed summary by multiple parameters
multi_summary = results_df.groupby(['RPM', 'WindMagnitude', 'AngleDegrees', 'AzimuthAngle'])[['Fx', 'Fy', 'Fz', 'Qx', 'Qy', 'Qz']].mean()
multi_summary.to_csv('detailed_multi_parameter_summary.csv')

print("Summary files created.")
print("All simulations completed successfully!")

In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import VLM
from wind import WindField
from scipy.interpolate import UnivariateSpline, LinearNDInterpolator, NearestNDInterpolator
from scipy.io import savemat
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numba as nb
import copy
import pyvista as pv
import pandas as pd

# Initialize and generate mesh as before
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
print('Propeller Loaded Succesfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Succesfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg

inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates



# Initialize the UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)

wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

time_points = []
normalized_forces = []
rho = 1.071778   
# 

mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)

rpm = 45
zerp = np.array([0, 0, 0])
com_position = np.array([300, 135, 50])
wind_func = WindField.update_wind_function(wind_field, com_position)
wind_velocity = -wind_func(zerp)


# Initialize arrays to store results
x_positions = []
torque = []
force = []

# Initial constants
y_position = 135
z_position = 50

n_revs = 2
n_steps_rev = 80
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (rpm * n_steps_rev)

omega_dict = {  # Motor speed initialization
    'Propeller_1': np.array([0, 0, rpm]),
    'Propeller_2': np.array([0, 0, -rpm]),
    'Propeller_3': np.array([0, 0, -rpm]),
    'Propeller_4': np.array([0, 0, rpm])
}
time_step = 1
for x_position in range(150, 451, 5):
    com_position = np.array([x_position, y_position, z_position])
    # Step 1: Calculate forces and moments using UVLM
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body, 
        omega=omega_dict,
        wind_field=wind_field,
        com_position=com_position,
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )

    # print("Forces:", forces_and_moments['total_force'])
    # print("Forces:", forces_and_moments['total_moment'])
    # print('force', forces_and_moments['Propeller_1']['force'])
    # print('momnet', forces_and_moments['Propeller_1']['moment'])

    x_positions.append(x_position)
    torque.append(forces_and_moments['Propeller_1']['moment'])
    force.append(forces_and_moments['Propeller_1']['force'])

    print("Time step:", x_position)
   
# Convert lists to numpy arrays
x_positions = np.array(x_positions)
torque = np.array(torque)
force = np.array(force)

# Method 1: Save as separate numpy arrays
np.save('x_positions.npy', x_positions)
np.save('torque_data.npy', torque)
np.save('force_data.npy', force)

# Method 2: Save as CSV files
# Torque CSV (assuming 3 components: Mx, My, Mz)
if torque.ndim >= 2 and torque.shape[1] >= 3:
    torque_df = pd.DataFrame({
        'x_position': x_positions,
        'torque_x': torque[:, 0],
        'torque_y': torque[:, 1],
        'torque_z': torque[:, 2]
    })
    torque_df.to_csv('torque_data.csv', index=False)

# Force CSV (assuming 3 components: Fx, Fy, Fz)
if force.ndim >= 2 and force.shape[1] >= 3:
    force_df = pd.DataFrame({
        'x_position': x_positions,
        'force_x': force[:, 0],
        'force_y': force[:, 1],
        'force_z': force[:, 2]
    })
    force_df.to_csv('force_data.csv', index=False)






In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import VLM
from wind import WindField
from scipy.interpolate import UnivariateSpline, LinearNDInterpolator, NearestNDInterpolator
from scipy.io import savemat
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numba as nb
import copy
import pyvista as pv
import pandas as pd

# Initialize and generate mesh as before
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
print('Propeller Loaded Succesfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Succesfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg

inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates



# Initialize the UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)

wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

time_points = []
normalized_forces = []
rho = 1.071778   
# 

mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)

rpm = 45
zerp = np.array([0, 0, 0])
com_position = np.array([300, 135, 50])
wind_func = WindField.update_wind_function(wind_field, com_position)
wind_velocity = -wind_func(zerp)


# Initialize arrays to store results
x_positions = []
torque_data = {
    'Propeller_1': [],
    'Propeller_2': [],
    'Propeller_3': [],
    'Propeller_4': []
}
force_data = {
    'Propeller_1': [],
    'Propeller_2': [],
    'Propeller_3': [],
    'Propeller_4': []
}
moment_data = {  # Add this new dictionary to store 'moment' data
    'Propeller_1': [],
    'Propeller_2': [],
    'Propeller_3': [],
    'Propeller_4': []
}

# Initial constants
y_position = 135
z_position = 50

n_revs = 2
n_steps_rev = 80
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (rpm * n_steps_rev)

omega_dict = {  # Motor speed initialization
    'Propeller_1': np.array([0, 0, rpm]),
    'Propeller_2': np.array([0, 0, -rpm]),
    'Propeller_3': np.array([0, 0, -rpm]),
    'Propeller_4': np.array([0, 0, rpm])
}
time_step = 1
for x_position in range(150, 451, 5):
    com_position = np.array([x_position, y_position, z_position])
    
    # Step 1: Calculate forces and moments using UVLM
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body, 
        omega=omega_dict,
        wind_field=wind_field,
        com_position=com_position,
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )

    x_positions.append(x_position)
    
    # Store data for all four propellers
    for prop_name in ['Propeller_1', 'Propeller_2', 'Propeller_3', 'Propeller_4']:
        torque_data[prop_name].append(forces_and_moments[prop_name]['moment'])
        force_data[prop_name].append(forces_and_moments[prop_name]['force'])
        moment_data[prop_name].append(forces_and_moments[prop_name]['total_body_moment'])  # Add this line

    print("Time step:", x_position)

# Convert all data to numpy arrays
for prop_name in ['Propeller_1', 'Propeller_2', 'Propeller_3', 'Propeller_4']:
    torque_data[prop_name] = np.array(torque_data[prop_name])
    force_data[prop_name] = np.array(force_data[prop_name])
    moment_data[prop_name] = np.array(moment_data[prop_name])  # Add this line

# Method 1: Save as numpy arrays
for prop_name in ['Propeller_1', 'Propeller_2', 'Propeller_3', 'Propeller_4']:
    np.save(f'{prop_name}_torque_data.npy', torque_data[prop_name])
    np.save(f'{prop_name}_force_data.npy', force_data[prop_name])
    np.save(f'{prop_name}_moment_data.npy', moment_data[prop_name])  # Add this line

# Save x_positions once
np.save('x_positions.npy', x_positions)

# Method 2: Save as CSV files
for prop_name in ['Propeller_1', 'Propeller_2', 'Propeller_3', 'Propeller_4']:
    # Torque CSV (total_body_moment)
    if torque_data[prop_name].ndim >= 2 and torque_data[prop_name].shape[1] >= 3:
        torque_df = pd.DataFrame({
            'x_position': x_positions,
            'total_body_moment_x': torque_data[prop_name][:, 0],
            'total_body_moment_y': torque_data[prop_name][:, 1],
            'total_body_moment_z': torque_data[prop_name][:, 2]
        })
        torque_df.to_csv(f'{prop_name}_torque_data.csv', index=False)
    
    # Force CSV
    if force_data[prop_name].ndim >= 2 and force_data[prop_name].shape[1] >= 3:
        force_df = pd.DataFrame({
            'x_position': x_positions,
            'force_x': force_data[prop_name][:, 0],
            'force_y': force_data[prop_name][:, 1],
            'force_z': force_data[prop_name][:, 2]
        })
        force_df.to_csv(f'{prop_name}_force_data.csv', index=False)
    
    # Moment CSV (moment)
    if moment_data[prop_name].ndim >= 2 and moment_data[prop_name].shape[1] >= 3:
        moment_df = pd.DataFrame({
            'x_position': x_positions,
            'moment_x': moment_data[prop_name][:, 0],
            'moment_y': moment_data[prop_name][:, 1],
            'moment_z': moment_data[prop_name][:, 2]
        })
        moment_df.to_csv(f'{prop_name}_moment_data.csv', index=False)



In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import VLM
from wind import WindField
from scipy.interpolate import UnivariateSpline, LinearNDInterpolator, NearestNDInterpolator
from scipy.io import savemat
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numba as nb
import copy
import pyvista as pv


# Initialize and generate mesh as before
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
print('Propeller Loaded Succesfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Succesfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg

inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates



# Initialize the UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)

wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

time_points = []
normalized_forces = []
rho = 1.071778   
# 

mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)

rpm = 45
zerp = np.array([0, 0, 0])
com_position = np.array([300, 135, 50])
wind_func = WindField.update_wind_function(wind_field, com_position)
wind_velocity = -wind_func(zerp)


# Initialize arrays to store results
x_positions = []
wind_velocities = []

# Initial constants
zerp = np.array([0, 0, 0])
y_position = 135
z_position = 50

omega_dict = {  # Motor speed initialization
    'Propeller_1': np.array([0, 0, -rpm]),
    'Propeller_2': np.array([0, 0, -rpm]),
    'Propeller_3': np.array([0, 0, -rpm]),
    'Propeller_4': np.array([0, 0, rpm])
}


R = 2.809 

# propeller_offset = np.array([1.35 * R, 1.35 * R, 0.25 * R])
# propeller_offset = np.array([1.35 * R, -1.35 * R, 0.25 * R])
# propeller_offset = np.array([-1.35 * R, 1.35 * R, 0.6 * R])
propeller_offset = np.array([-1.35 * R, -1.35 * R, 0.6 * R])

# Loop through x positions from 150 to 450 in steps of 5
for x_position in range(150, 451, 5):  # 451 to include 450
    # Create com_position with current x value
    com_position_base = np.array([x_position, y_position, z_position])
    com_position = com_position_base + propeller_offset
    
    # Calculate wind function and velocity
    wind_func = WindField.update_wind_function(wind_field, com_position)
    wind_velocity = -wind_func(zerp)
    
    # Store results
    x_positions.append(x_position)
    wind_velocities.append(wind_velocity)
    
    # Optional: Print progress (can be removed)
    print(f"x: {x_position}, wind_velocity: {wind_velocity}")
x_positions = np.array(x_positions)
# Method 2: Save as CSV
# Convert list of wind velocities to a proper 2D numpy array
if wind_velocities:
    wind_velocities = np.array(wind_velocities)
    
    # Check if wind_velocities has the right shape
    if wind_velocities.ndim == 1:
        # If it's 1D, reshape or split into components
        # Assuming wind_velocity is a 3-component vector
        wind_velocities = wind_velocities.reshape(-1, 3)
else:
    print("Warning: No wind velocities were collected!")

# Example: Save to file (choose one of these methods)
# Method 1: Save as numpy arrays
np.save('x_positions.npy', x_positions)
np.save('wind_velocities.npy', wind_velocities)

# Method 2: Save as CSV
import pandas as pd

# Create DataFrame depending on the shape of wind_velocities
if wind_velocities.ndim >= 2 and wind_velocities.shape[1] >= 3:
    df = pd.DataFrame({
        'x_position': x_positions,
        'wind_velocity_x': wind_velocities[:, 0],
        'wind_velocity_y': wind_velocities[:, 1],
        'wind_velocity_z': wind_velocities[:, 2]
    })
else:
    # Fallback if wind_velocities doesn't have expected shape
    df = pd.DataFrame({
        'x_position': x_positions,
        'wind_velocity': wind_velocities.flatten()
    })

df.to_csv('Propeller_4_wind_velocities.csv', index=False)

print('wind',wind_velocity )
n_revs = 2
n_steps_rev = 80
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (rpm * n_steps_rev)


for time_step in range(1, 2):

    # Step 1: Calculate forces and moments using UVLM
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body, 
        omega=omega_dict,
        wind_field=wind_field,
        com_position=com_position,
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )



    # print("Forces:", forces_and_moments['total_force'])
    # print("Forces:", forces_and_moments['total_moment'])
    print('force', forces_and_moments['Propeller_1']['force'])
    print('momnet', forces_and_moments['Propeller_1']['moment'])

    print("Time step:", time_step)
   




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import matplotlib.patches as mpatches

# Create a visual representation of the deduction logic
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Top plot: Motor torque analysis
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 6)
ax1.axis('off')

# Title
ax1.text(5, 5.5, 'Flight Control Deduction Logic', fontsize=20, ha='center', fontweight='bold')

# Benchmark box
benchmark_box = FancyBboxPatch((0.5, 4), 2, 1, boxstyle="round,pad=0.1", 
                             facecolor='lightblue', edgecolor='black', linewidth=2)
ax1.add_patch(benchmark_box)
ax1.text(1.5, 4.5, 'No-Load Benchmark\n-1381.99 N·m\n@45 RPM', ha='center', va='center', fontweight='bold')

# Motor analysis boxes
motors = ['Motor 1\n(Front)', 'Motor 2\n(Right)', 'Motor 3\n(Back)', 'Motor 4\n(Left)']
colors = ['lightgreen', 'lightcoral', 'lightgreen', 'lightcoral']
positions = [(3, 4), (5, 4.5), (7, 4), (5, 3.5)]

for i, (motor, color, pos) in enumerate(zip(motors, colors, positions)):
    box = FancyBboxPatch((pos[0]-0.7, pos[1]-0.4), 1.4, 0.8, boxstyle="round,pad=0.1",
                        facecolor=color, edgecolor='black', linewidth=1)
    ax1.add_patch(box)
    ax1.text(pos[0], pos[1], motor, ha='center', va='center', fontsize=10)
    
    # Add arrows showing wind effect
    if i in [0, 2]:  # Front and Back motors
        ax1.text(pos[0], pos[1]-0.2, 'More Negative\n(+AoA)', ha='center', va='center', fontsize=8)
    else:  # Left and Right motors
        ax1.text(pos[0], pos[1]-0.2, 'Less Negative\n(-AoA)', ha='center', va='center', fontsize=8)

# Add arrows showing relationships
arrow_props = dict(arrowstyle='->', connectionstyle='arc3,rad=0.1', color='black', lw=1.5)
for pos in positions:
    ax1.annotate('', xy=(pos[0], pos[1]-0.4), xytext=(2.5, 4.5), arrowprops=arrow_props)

# Add IMU data
imu_box = FancyBboxPatch((8.5, 4), 1.5, 1, boxstyle="round,pad=0.1", 
                        facecolor='lightyellow', edgecolor='black', linewidth=2)
ax1.add_patch(imu_box)
ax1.text(9.25, 4.5, 'IMU Data\n• Roll\n• Pitch\n• Yaw', ha='center', va='center', fontweight='bold')

# Bottom plot: Deduction flow
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 8)
ax2.axis('off')

# Create deduction flow
deduction_steps = [
    "1. Motor Torque Analysis\n• Front/Back: More resistance (headwind)\n• Left/Right: Less resistance (crosswind)",
    "2. Thrust Implications\n• Front/Back: Reduced thrust\n• Left/Right: Normal/increased thrust\n• Net effect: Pitch moment",
    "3. IMU Correlation\n• Pitch angle matches torque pattern\n• Roll shows left/right asymmetry\n• Yaw indicates rotation tendency",
    "4. Wind Direction Deduction\n• Primary headwind component\n• Secondary crosswind component\n• Overall wind vector direction",
    "5. Control Response\n• Adjust front/back motor RPM\n• Compensate roll with left/right\n• Stabilize yaw with differential torque"
]

y_positions = [7, 5.5, 4, 2.5, 1]
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral', 'lightgray']

for i, (step, y_pos, color) in enumerate(zip(deduction_steps, y_positions, colors)):
    box = FancyBboxPatch((1, y_pos-0.6), 8, 1.2, boxstyle="round,pad=0.2",
                        facecolor=color, edgecolor='black', linewidth=1)
    ax2.add_patch(box)
    ax2.text(5, y_pos, step, ha='center', va='center', fontsize=11, wrap=True)
    
    if i < len(deduction_steps) - 1:
        ax2.arrow(5, y_pos-0.6, 0, -0.2, head_width=0.2, head_length=0.1, fc='black', ec='black')

plt.tight_layout()
plt.show()

# Now let's create a specific example analysis
fig, ax = plt.subplots(figsize=(12, 8))

# Example scenario
scenario_data = {
    'Motor 1 (Front)': {'torque': -1450, 'benchmark_diff': -68.01, 'status': 'More Negative', 'thrust': 'Reduced', 'color': 'red'},
    'Motor 2 (Right)': {'torque': -1320, 'benchmark_diff': 61.99, 'status': 'Less Negative', 'thrust': 'Normal/Increased', 'color': 'green'},
    'Motor 3 (Back)': {'torque': -1470, 'benchmark_diff': -88.01, 'status': 'More Negative', 'thrust': 'Reduced', 'color': 'red'},
    'Motor 4 (Left)': {'torque': -1340, 'benchmark_diff': 41.99, 'status': 'Less Negative', 'thrust': 'Normal/Increased', 'color': 'green'}
}

imu_data = {
    'Pitch': '+5° (nose up)',
    'Roll': '+2° (right wing down)',
    'Yaw': '-1° (slight left turn)'
}

# Create the analysis visualization
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Title
ax.text(5, 9.5, 'Example: Wind Direction Deduction', fontsize=18, ha='center', fontweight='bold')

# Motor status boxes
motor_positions = {
    'Motor 1 (Front)': (5, 7),
    'Motor 2 (Right)': (7, 5),
    'Motor 3 (Back)': (5, 3),
    'Motor 4 (Left)': (3, 5)
}

for motor, data in scenario_data.items():
    pos = motor_positions[motor]
    box = FancyBboxPatch((pos[0]-1, pos[1]-0.5), 2, 1, boxstyle="round,pad=0.1",
                        facecolor=data['color'], alpha=0.3, edgecolor='black', linewidth=1)
    ax.add_patch(box)
    
    text = f"{motor}\nTorque: {data['torque']} N·m\nDiff: {data['benchmark_diff']:.1f}\n{data['status']}\nThrust: {data['thrust']}"
    ax.text(pos[0], pos[1], text, ha='center', va='center', fontsize=9)

# IMU data box
imu_box = FancyBboxPatch((7.5, 7), 2, 2, boxstyle="round,pad=0.1",
                        facecolor='lightyellow', edgecolor='black', linewidth=2)
ax.add_patch(imu_box)
ax.text(8.5, 8, 'IMU Data', ha='center', fontweight='bold', fontsize=12)
imu_text = '\n'.join([f"{k}: {v}" for k, v in imu_data.items()])
ax.text(8.5, 7.3, imu_text, ha='center', va='center', fontsize=10)

# Wind direction deduction
deduction_box = FancyBboxPatch((1, 1), 8, 1.5, boxstyle="round,pad=0.2",
                              facecolor='lightblue', edgecolor='black', linewidth=2)
ax.add_patch(deduction_box)

deduction_text = """DEDUCTION:
• Front/Back motors show increased resistance → Headwind component
• Left/Right motors show decreased resistance → Crosswind component  
• IMU pitch (+5°) confirms nose-up moment from headwind
• IMU roll (+2°) suggests right crosswind component
• Combined wind direction: ~30° from front-right (NE wind if drone faces North)"""

ax.text(5, 1.75, deduction_text, ha='center', va='center', fontsize=11)

plt.tight_layout()
plt.show()

# Create a summary of what can be deduced
deductions = {
    "Wind Direction": [
        "Primary component direction (headwind/tailwind)",
        "Secondary component (crosswind)",
        "Overall wind vector and magnitude"
    ],
    "Vehicle State": [
        "Current stability margins",
        "Control response effectiveness",
        "Dynamic response to disturbances"
    ],
    "Control Actions": [
        "Motor RPM adjustments needed",
        "Control surface deflections (if present)",
        "Predicted flight path corrections"
    ],
    "Performance Monitoring": [
        "Actual thrust vs commanded thrust",
        "Power efficiency under wind loads",
        "Battery consumption adjustments"
    ]
}

print("=== What Can Be Deduced from Combined Data ===\n")
for category, items in deductions.items():
    print(f"{category}:")
    for item in items:
        print(f"  • {item}")
    print()

print("=== Control Strategy Example ===")
print("Based on the scenario above:")
print("1. Increase front/back motor RPM to compensate for thrust loss")
print("2. Slightly reduce right motor RPM to counter roll moment")
print("3. Apply differential torque to counteract yaw tendency")
print("4. Adjust control gains based on wind magnitude estimation")
print("5. Update wind disturbance model for feed-forward control")

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Read the CSV file
df = pd.read_csv('rpm_wind_simulation_results_correct_3.csv')

# Extract the input (features) and output (target)
X = df[['RPM', 'Propeller_1_Qz']].values          # Independent variables
y = df['WindZ'].values         # Dependent variable

# Choose polynomial degree
degree = 3  # You can try 3, 4, etc. for more complexity

# Generate polynomial features
poly = PolynomialFeatures(degree)
X_poly = poly.fit_transform(X)

# Fit the polynomial regression model
model = LinearRegression()
model.fit(X_poly, y)

# Predict using the model
y_pred = model.predict(X_poly)

# Evaluate the fit
r2 = r2_score(y, y_pred)
print(f"R² score: {r2:.4f}")

# Get the names of the polynomial terms
feature_names = poly.get_feature_names_out(['RPM', 'Propeller_1_Qz'])

# Construct the polynomial equation
equation = "y = " + f"{model.intercept_:.6f}"  # Start with the intercept term
for name, coef in zip(feature_names[1:], model.coef_[1:]):  # Skip the first term as it's the intercept
    equation += f" + ({coef:.6f}) * {name}"

# Print the polynomial equation
print(f"Polynomial Equation: {equation}")


R² score: 0.9991
Polynomial Equation: y = 13.195318 + (-1.237085) * RPM + (-0.033724) * Propeller_1_Qz + (0.019313) * RPM^2 + (0.000680) * RPM Propeller_1_Qz + (-0.000001) * Propeller_1_Qz^2 + (-0.000149) * RPM^3 + (-0.000005) * RPM^2 Propeller_1_Qz + (0.000000) * RPM Propeller_1_Qz^2 + (-0.000000) * Propeller_1_Qz^3


In [5]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('motor_simulation_log.csv')

# Extract Motor Speed and Current
motor_speed = df['Motor_Speed_rad_s'].values  # omega
current = df['Current_A'].values
time = df['Time_s'].values

# Constants
J_rot = 0.064         # kg·m²
I_r = 274.69          # kg·m²
r = 18.75             # Gear ratio
K_Q = 0.463           # Nm/A
total_inertia = J_rot * r**2 + I_r

# Calculate motor torque
motor_torque = current * K_Q  # Q_m

# Calculate rotor speed from motor speed
rotor_speed = motor_speed / r  # Omega

# Calculate aerodynamic torque
dt = 0.01  # Time step from the simulation
dOmega_dt = np.gradient(rotor_speed, dt)
aero_torque = total_inertia * dOmega_dt - motor_torque * r

# Create DataFrame with just Qaero and RotorSpeed
output_data = pd.DataFrame({
    'Time_s': time,
    'RotorSpeed_rad_s': rotor_speed,
    'Qaero_Nm': aero_torque
})

# Save to CSV file
output_data.to_csv('Qaero_RotorSpeed.csv', index=False)
print(f"Data saved to 'Qaero_RotorSpeed.csv'")
print(f"Number of data points: {len(output_data)}")

# Show first few rows to verify
print("\n=== First 10 rows of saved data ===")
print(output_data.head(10))

# Show summary statistics
print("\n=== Summary Statistics ===")
print(f"Average Rotor Speed: {np.mean(rotor_speed):.4f} rad/s")
print(f"Average Aerodynamic Torque: {np.mean(aero_torque):.1f} Nm")
print(f"Data range: {time[0]:.2f}s to {time[-1]:.2f}s")
print(f"Total time points: {len(time)}")

KeyError: 'Motor_Speed_rad_s'